## Data Scraping - tripadvisor.com

In [ ]:
#pip install tqdm

In [2]:
import re
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import feedparser

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC   # acciones encadenadas
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options



import tqdm
from bs4 import BeautifulSoup
import time
import re
import json

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../src')

from pymongo import MongoClient

In [3]:
options=Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-allow-origins=*")
options.add_argument("user-data-dir=cookies")
#options.add_argument('--headless')                 #Habilitar si no queremos ver la ventana
options.add_experimental_option("detach", True)    #Esta opción corrige el error de cierre repentino
options.add_argument('--start-minimized')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-extensions')
options.add_argument('--disable-infobars')

In [3]:
driver = webdriver.Chrome(options=options)    # abre una ventana de chrome 

driver.get('https://www.tripadvisor.es/FindRestaurants?geo=187514&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true')

Tras importar las librerías necesarias y ejecutar las opciones, doy paso a retirar las cookies del driver que voy a utilizar. 
- Al habilitar la opcion que mantiene las cookies eliminadas, solamente ejecutamos el siguiente webdriver igualando options a options y ya nos mantendrá el driver sin las cookies.

#### 3. Empezamos el scrapeo

Creo una funcion que me vale para la 

In [5]:
def extraer(url):

    global table

    while True:
        driver.get(url)
        time.sleep(4)

        print('entro')

        # paso 2: cookies
        time.sleep(3)

        try:
            aceptar = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
            aceptar.click()

        except Exception as e:
            print(f'no salen las cookies')

        # paso 3: obtener tabla

        time.sleep(4)

        print('0')

        cartas = []

        try:
            for e in range(len(driver.find_elements(By.XPATH, './/div[@class="yJIls z y M0"]'))):
                box = driver.find_elements(By.XPATH, './/div[@class="yJIls z y M0"]')[e].text.split('\n')
                cartas.append(box)

            print('1')

        except Exception as e:
            print(f'Algo falló en el paso 3: {e}')

        # paso 4: concatenamos las tablas
        if cartas:
            table += cartas
            break  

        else:
            print('La lista "cartas" está vacía, se vuelve a abrir la URL.')

Hago una selección del tipo de establecimiento en tripadvisor, y de aquí saco algunas de las url que me interesan. 
- Intento crear una muestra.  

In [ ]:
url1 ='https://www.tripadvisor.es/FindRestaurants?geo=187514&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url2 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=30&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

In [17]:
url3 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=60&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url4 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=60&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url5 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=90&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

In [18]:

url6 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=120&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url7 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=150&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url8 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=180&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url9 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=210&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url10 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=240&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

url11 = 'https://www.tripadvisor.es/FindRestaurants?geo=187514&offset=270&cuisines=10655%2C10649%2C10654%2C10642%2C10640%2C10659%2C4617%2C10670%2C10679%2C10646%2C10648%2C9908%2C10671%2C5473%2C10641%2C10653%2C10639%2C5379%2C5110%2C10345%2C10669%2C10643%2C10651%2C21353%2C10715%2C10668%2C10686%2C10676%2C10682%2C20076%2C10683%2C10346%2C5086%2C21367%2C10763%2C10621%2C21355%2C20062%2C20069%2C10661%2C10347%2C10717&broadened=true'

In [6]:
table = []

extraer(url1)
extraer(url2)

entro
no salen las cookies
0
1
entro
no salen las cookies
0
1


In [ ]:
data = pd.DataFrame(table)

In [13]:
data.to_csv('./datos/data1.csv', index=False, encoding='utf-8')

In [19]:
table =[]

extraer(url3)
extraer(url4)
extraer(url5)
extraer(url6)
extraer(url7)
extraer(url8)
extraer(url9)
extraer(url10)
extraer(url11)

entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1
entro
no salen las cookies
0
1


In [22]:
data2 = pd.DataFrame(table)

In [23]:
data2.tail()

,0,1,2,3,4,5,6,7,8,9,10
274,296. Mad by Robleño,35 opinionesAbierto ahora,"Mediterránea, Española€€ - €€€",Para repetir.,¡Deliciosa cocina casera!,None,None,None,None,None,None
275,297. El Cunzato,31 opiniones,"Italiana, Mediterránea€",¡Gran descubrimiento!,Comida y personal TOP!,Hacer pedido online,None,None,None,None,None
276,298. Crudo,13 opinionesAbierto ahora,"Mediterránea, Saludable€€ - €€€Carta",Muy recomendable,Excelente,Hacer pedido online,None,None,None,None,None
277,299. La Lonja Restaurante,550 opinionesCierra en 4 minutos,"Marisco, Internacional€€ - €€€Carta",Agradable sitio en un lugar privilegiado,Excelente Experiencia,Reservar,None,None,None,None,None
278,300. Federica By Chef Oswaldo Sanchez,72 opiniones,"Italiana, Latina€€ - €€€",BUENÍSIMO,Federica\Cachaoeichin,None,None,None,None,None,None


In [21]:
data2.to_csv('./datos/data2.csv', index=False, encoding='utf-8')

In [24]:
#driver.quit()